###### This file is to test the bass_biquad function and compare its result with SoX and MATLAB audio toolbox.

The result summary:

(1) The biquad coefficients calculated from bass_biquad and Matlab toolbox are the same (error < 1e-10).

(2) The output waveform error (absolute value difference) between SoX and bass_biquad is 1e-3 for unormalized coefficients and 1.4e-4 for normalized coefficients.

(3) When using `lfilter` function in `torchaudio/functional.py`, I input two groups of biquad coefficient parameters (one group is normalized by a0, and the other is unnormalized). The maximum error between two output waveforms is 6e-4.


Test options and results:

1) Compare biquad coefficients calculated by bass_biquad function and website calculator. Lowpass, highpass, bandpass, low shelf and high shelf biquad filters are tested to check if their coefficients match with website calculator.

Result: The coefficients of lowpass, highpass and bandpass filters match with website calculator's result (error < 1e-10). The coefficients of low shelf and high shelf filters don't match with website's result. The reason is that in the website calculator, the coefficients are independent of input Q value for low and high shelf filters. The calculator has a default slope parameter S and then calculates its own Q value. In my code, Q value is a variable and S will be calculated accordingly.

website calulator: https://www.earlevel.com/main/2013/10/13/biquad-calculator-v2/

2) Calculate a simple corner case by hand to check if coefficients calculated by bass_biquad are correct:
sample_rate = 44100, Central_frequency = 11025, Q = 1, Gain = 6

Result: The calculation results by hand match with bass_biquad function results.

3) Compare bass_biquad function with MATLAB audio system toolbox

Result: The calculated coefficient result are exactly the same with MATLAB audio system toolbox (error < 1e-10).

For treble biquad filter:
Coefficients from bass_biquad:
`[b0, b1, b2, a1, a2] =
[1.1728093663351498 1.115899098850206 0.26024700972986287 1.195000148261586 0.3539553266536329]`

Coefficients from MATLAB toolbox:
`[b0, b1, b2, a1, a2] =
[1.172809366335150 1.115899098850206 0.260247009729863 1.195000148261586 0.353955326653633]`

For bass biquad filter:
Coefficients from bass_biquad:
`[b0, b1, b2, a1, a2] =
[0.5877970818856679 0.5592743823599009 0.13043247886627984 1.1950001482615862 0.35395532665363294]`

From matlab toolbox:
`[b0, b1, b2, a1, a2] =
[0.587797081885668 0.559274382359901 0.13043247886627980 1.195000148261586 0.353955326653633]`


MATLAB audio system toolbox: https://www.mathworks.com/help/audio/ref/designshelvingeq.html

4) Check SoX implementation with different input parameters (slope S, Q, and coefficients calculated by bass_biquad function)

Result: For three input parameters, output waveforms are the same (error < 1e-10), which means the calculated coefficients from bass_biquad function and SoX uses are the same.

`abs(SoX_output_waveform (s) - SoX_output_waveform (code's biquad coefficients)) < 1e-10
abs(SoX_output_waveform (q) - SoX_output_waveform (code's biquad coefficients)) < 1e-10`


Sox link: http://sox.sourceforge.net/sox.html
https://www.w3.org/2011/audio/audio-eq-cookbook.html

5) Compare output waveform error between code and sox for bass biquad filter. 

Result: 
The maximum error is at the level of 1e-3~1e-4 at edge cases:
sample rate = 44100, s = 1, gain = 40, q = 0.707, fc = 1000.

`abs(SoX_output_waveform - bass_biquad_ouput_waveform) ~ 1e-3`

6) Compare result with and without coefficients normalization.

Result:
Compare to the error with SoX without coefficient normalization, the max error in bass_biquad function after coefficient normalization becomes smaller (from 1e-3 to 1.4e-4).

Without normalization:
`output_waveform1 = F.biquad(waveform, b0, b1, b2, a0, a1, a2);`

With normalization:
`output_waveform2 = F.biquad(waveform, b0/a0, b1/a0, b2/a0, 1, a1/a0, a2/a0)`

`abs(output_waveform1 - output_waveform2) ~ 1e-4`

7) Compare output waveforms after bass biquad filter between SoX, code and MATLAB toolbox

Result:
Three outputs are very close. In MATLAB toolbox, the max gain that can be set as 12 db, in this case, the max errors between three outputs are at the level of 1e-5.

`abs(output_waveform_code - output_waveform_SoX) ~ 1e-5
abs(output_waveform_code - output_waveform_Matlab) ~ 1e-5`

In [39]:
import torch 
import torchaudio
from torch import Tensor
import torchaudio.functional_update as F
import common_utils
import matplotlib.pyplot as plt
import IPython.display as ipd
import math

### 1. Compare coeffients by bass_biquad function and website calculator

##### 1.1 Check lowpass coefficients by website calculator and bass_biquad calculation 

In [2]:
central_freq = 1000
Q = .707
gain = 6

filename = 'steam-train-whistle-daniel_simon.mp3'
#filename = 'sinewave.wav'
noise_filepath = common_utils.get_asset_path(filename)
waveform, sample_rate = torchaudio.load(noise_filepath, normalization=True)

In [3]:
def lowpass_biquad(
        waveform: Tensor,
        sample_rate: int,
        cutoff_freq: float,
        Q: float = 0.707
) -> Tensor:
    r"""Design biquad lowpass filter and perform filtering.  Similar to SoX implementation.
    Args:
        waveform (torch.Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        cutoff_freq (float): filter cutoff frequency
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``)
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    """
    w0 = 2 * math.pi * cutoff_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q

    b0 = (1 - math.cos(w0)) / 2
    b1 = 1 - math.cos(w0)
    b2 = b0
    a0 = 1 + alpha
    a1 = -2 * math.cos(w0)
    a2 = 1 - alpha
    
    print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)
    
    #return biquad(waveform, b0, b1, b2, a0, a1, a2)

In [4]:
lowpass_biquad(waveform, sample_rate, central_freq, Q)

0.004603935028493071 0.009207870056986141 0.004603935028493071 -1.799071616595651 0.8174873567096231


a0 = 0.00460393502849307
a1 = 0.00920787005698614
a2 = 0.00460393502849307
b1 = -1.7990716165956508
b2 = 0.8174873567096232

Matched!  and try another set to double check

In [5]:
sample_rate = 5000
Q = 2
lowpass_biquad(waveform, sample_rate, central_freq, Q)

0.27912547689603473 0.5582509537920695 0.27912547689603473 -0.4993148324732646 0.6158167400574035


`a0 = 0.27912547689603473
a1 = 0.5582509537920695
a2 = 0.27912547689603473
b1 = -0.49931483247326475
b2 = 0.6158167400574036`


Matched again!

##### 1.2 Check highpass coefficients by website calculator and bass_biquad calculation

In [6]:
central_freq = 10000
Q = 1
gain = 6
sample_rate = 44100

In [7]:
def highpass_biquad(
        waveform: Tensor,
        sample_rate: int,
        cutoff_freq: float,
        Q: float = 0.707
) -> Tensor:
    r"""Design biquad highpass filter and perform filtering.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        cutoff_freq (float): filter cutoff frequency
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``)
    Returns:
        Tensor: Waveform dimension of `(..., time)`
    """
    w0 = 2 * math.pi * cutoff_freq / sample_rate
    alpha = math.sin(w0) / 2. / Q

    b0 = (1 + math.cos(w0)) / 2
    b1 = -1 - math.cos(w0)
    b2 = b0
    a0 = 1 + alpha
    a1 = -2 * math.cos(w0)
    a2 = 1 - alpha
    
    print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)

    
    #return F.biquad(waveform, b0, b1, b2, a0, a1, a2)

In [8]:
highpass_biquad(waveform, sample_rate, central_freq, Q)

0.38319940325388924 -0.7663988065077785 0.38319940325388924 -0.19471651170319731 0.3380811013123595


`a0 = 0.3831994032538892
a1 = -0.7663988065077784
a2 = 0.3831994032538892
b1 = -0.1947165117031973
b2 = 0.33808110131235947
`
Matched, tried another set and also matched !

##### 1.3 Check bandpass coefficients by website calculator and bass_biquad calculation 

In [9]:
def bandpass_biquad(
        waveform: Tensor,
        sample_rate: int,
        central_freq: float,
        Q: float = 0.707,
        const_skirt_gain: bool = False
) -> Tensor:
    r"""Design two-pole band-pass filter.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        central_freq (float): central frequency (in Hz)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``)
        const_skirt_gain (bool, optional) : If ``True``, uses a constant skirt gain (peak gain = Q).
            If ``False``, uses a constant 0dB peak gain. (Default: ``False``)
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q

    temp = math.sin(w0) / 2 if const_skirt_gain else alpha
    b0 = temp
    b1 = 0.
    b2 = -temp
    a0 = 1 + alpha
    a1 = -2 * math.cos(w0)
    a2 = 1 - alpha
    
    print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)

    #return biquad(waveform, b0, b1, b2, a0, a1, a2)


In [10]:
central_freq = 10000
Q = 1
gain = 6
sample_rate = 44100

In [11]:
bandpass_biquad(waveform, sample_rate, central_freq, Q)

0.33095944934382027 0.0 -0.33095944934382027 -0.19471651170319731 0.3380811013123595


`a0 = 0.3309594493438202
a1 = 0
a2 = -0.3309594493438202
b1 = -0.1947165117031973
b2 = 0.33808110131235947
`

Matched, tried other set and also matched !

##### 1. 4 Check high shelf coefficients by website calculator and bass_biquad calculation

In [12]:
def treble_biquad(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 3000,
        Q: float = 0.707
) -> Tensor:
    r"""Design a treble tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``3000``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) + temp2 + temp1)
    b1 = -2 * A * ((A - 1) + temp3)
    b2 = A * ((A + 1) + temp2 - temp1)
    a0 = (A + 1) - temp2 + temp1
    a1 = 2 * ((A - 1) - temp3)
    a2 = (A + 1) - temp2 - temp1
    
    print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)

    #return biquad(waveform, b0, b1, b2, a0, a1, a2)

In [13]:
central_freq = 18000
gain = 6
sample_rate = 44100
Q = 1
#A = math.exp(gain / 40 * math.log(10))
#slope = 0.5
#Q = 1/math.sqrt(A + 2 + 1/A)

In [14]:
treble_biquad(waveform, sample_rate, gain, central_freq, Q)

1.1258190808574104 1.3346319052890663 0.5881376411661036 1.4292379358836353 0.6193506914289453


`a0 = 1.1729814928394098
a1 = 1.0932923261729204
a2 = 0.3865657418579309
b1 = 1.209579276550885
b2 = 0.4432602843193761
`

Not matched ! I found that in the calculator website, the coefficients are independent of Q value. It has a default slope parameter S that can calculate its own Q. And in our code, Q value is set and S can be calculated.

##### Check low shelf (bass) coefficients by website calculator and bass_biquad calculation

In [15]:
def bass_biquad(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 100,
        Q: float = 1
) -> Tensor:
    r"""Design a bass tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``100``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) - temp2 + temp1)
    b1 = 2 * A * ((A - 1) - temp3)
    b2 = A * ((A + 1) - temp2 - temp1)
    a0 = (A + 1) + temp2 + temp1
    a1 = -2 * ((A - 1) + temp3)
    a2 = (A + 1) + temp2 - temp1
    
    print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)

In [16]:
central_freq = 10000
Q = 1
gain = 6
sample_rate = 44100

In [17]:
bass_biquad(waveform, sample_rate, gain, central_freq, Q)

1.3660529664289085 0.047595611249911476 0.4555582896232585 -0.4185965129510136 0.355419131851242


`a0 = 1.4199974570112823
a1 = 0.32913667880445635
a2 = 0.25714197787478704
b1 = -0.17124139038429181
b2 = 0.1767613656973212
`

Not matched ! Same reason as mentioned in treble filter

### 2. Compare bass_biquad function with MATLAB audio system toolbox

Now I find the Matlab audio toolbox that can also implement biquad filters. https://www.mathworks.com/help/audio/ref/designshelvingeq.html#d120e24679 

Here I compare the coefficients of code result and matlab result for treble and bass filters. Both are matched (error < 1e-10).

###### 2.1 Test the coefficients of treble (high-shelf) filter

In [18]:
central_freq = 18000
gain = 6
sample_rate = 44100

A = math.exp(gain / 40 * math.log(10))
slope = 0.5
Q = 1/math.sqrt(A + 2 + 1/A)

treble_biquad(waveform, sample_rate, gain, central_freq, Q)

1.1728093663351498 1.115899098850206 0.26024700972986287 1.195000148261586 0.3539553266536329


In matlab code:

`Fs = 44.1e3;
gain = 6;
slope = 0.5;
central_freq = 18000 

Fc = central_freq/(Fs/2);

[B,A] = designShelvingEQ(gain,slope,Fc,'hi');
`

result:
`1.172809366335150 1.115899098850206 0.260247009729863 1.195000148261586 0.353955326653633`
 
Matched! I also double check different sets and all are matched!

###### 2.2 Test the coefficients of bass (low-shelf) filter

In [19]:
def bass_biquad(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 100,
        Q: float = 1
) -> Tensor:
    r"""Design a bass tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``100``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) - temp2 + temp1)
    b1 = 2 * A * ((A - 1) - temp3)
    b2 = A * ((A + 1) - temp2 - temp1)
    a0 = (A + 1) + temp2 + temp1
    a1 = -2 * ((A - 1) + temp3)
    a2 = (A + 1) + temp2 - temp1
    
    print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)

    #return F.biquad(waveform, b0, b1, b2, a0, a1, a2)

In [20]:
central_freq = 18000
gain = -6
sample_rate = 44100

A = math.exp(gain / 40 * math.log(10))
slope = 0.5
Q = 1/math.sqrt(A + 2 + 1/A)

bass_biquad(waveform, sample_rate, gain, central_freq, Q)

0.5877970818856679 0.5592743823599009 0.13043247886627984 1.1950001482615862 0.35395532665363294


Matlab result:

Fs = 44.1e3;
gain = -6;
slope = 0.5;
central_freq = 18000 

Fc = central_freq/(Fs/2);

[B,A] = designShelvingEQ(gain,slope,Fc,'lo');

`
0.587797081885668 0.559274382359901 0.13043247886627980 1.195000148261586 0.353955326653633`

Matched! Different sets are also matched.

### 3. Check SoX implementation with different input parameters (slope S, Q, and coefficients calculated by bass_biquad function)

Firstly, I calculate the errors between SoX and bass_biquad results for treble and bass filters. For further analysis, the performance is measured by the count of elements with error larger than atol = 1e-4. 

Then I put slope, Q and biquad coefficients into SoX individually and see the result differences

In [21]:
def treble_biquad(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 3000,
        Q: float = 0.707
) -> Tensor:
    r"""Design a treble tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``3000``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) + temp2 + temp1)
    b1 = -2 * A * ((A - 1) + temp3)
    b2 = A * ((A + 1) + temp2 - temp1)
    a0 = (A + 1) - temp2 + temp1
    a1 = 2 * ((A - 1) - temp3)
    a2 = (A + 1) - temp2 - temp1
    return F.biquad(waveform, b0, b1, b2, a0, a1, a2)

    #return F.biquad(waveform, b0/a0, b1/a0, b2/a0, a0/a0, a1/a0, a2/a0)

In [22]:
central_freq = 1000
q = 0.707
gain = 40
#s = 0.5
#filename = 'steam-train-whistle-daniel_simon.mp3'
#filename = 'sinewave.wav'
filename = 'whitenoise.wav'
noise_filepath = common_utils.get_asset_path(filename)
E = torchaudio.sox_effects.SoxEffectsChain()
E.set_input_file(noise_filepath)
E.append_effect_to_chain("treble", [gain, central_freq, str(q) + 'q'])
sox_output_waveform, sr = E.sox_build_flow_effects()

waveform, sample_rate = torchaudio.load(noise_filepath, normalization=True)
output_waveform = treble_biquad(waveform, sample_rate, gain, central_freq, q)
#self.assertEqual(output_waveform, sox_output_waveform, atol=1e-4, rtol=1e-5)

In [23]:
# Test the treble biquad filter 
atol = 1e-4
diff = abs(output_waveform - sox_output_waveform)
print((diff > atol).sum())

tensor(0)


In [24]:
def bass_biquad(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 100,
        Q: float = 1
) -> Tensor:
    r"""Design a bass tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``100``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) - temp2 + temp1)
    b1 = 2 * A * ((A - 1) - temp3)
    b2 = A * ((A + 1) - temp2 - temp1)
    a0 = (A + 1) + temp2 + temp1
    a1 = -2 * ((A - 1) + temp3)
    a2 = (A + 1) + temp2 - temp1
    
    #print(b0/a0, b1/a0, b2/a0, a1/a0, a2/a0)
    return F.biquad(waveform, b0, b1, b2, a0, a1, a2)
    #return F.biquad(waveform, b0/a0, b1/a0, b2/a0, a0/a0, a1/a0, a2/a0)

In [25]:
central_freq = 1000
q = 0.707
gain = 40
#s = 0.5
#filename = 'steam-train-whistle-daniel_simon.mp3'
#filename = 'sinewave.wav'
filename = 'whitenoise.wav'
noise_filepath = common_utils.get_asset_path(filename)
E = torchaudio.sox_effects.SoxEffectsChain()
E.set_input_file(noise_filepath)
E.append_effect_to_chain("bass", [gain, central_freq, str(q) + 'q'])
sox_output_waveform, sr = E.sox_build_flow_effects()

waveform, sample_rate = torchaudio.load(noise_filepath, normalization=True)
output_waveform = bass_biquad(waveform, sample_rate, gain, central_freq, q)
#self.assertEqual(output_waveform, sox_output_waveform, atol=1e-4, rtol=1e-5)

In [26]:
# Test bass treble biquad filter 
atol = 1e-4
diff = abs(output_waveform - sox_output_waveform)
print((diff > atol).sum())

tensor(20161)


For bass filters, in order to check if SoX and my code have same coefficients, I input different inputs (q, s and biquad coefficients from bass_biquad function) to SoX and check the differences between the outpt waveforms.

In [27]:
def bass_biquad_coeff(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 100,
        Q: float = 1
) -> Tensor:
    r"""Design a bass tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``100``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) - temp2 + temp1)
    b1 = 2 * A * ((A - 1) - temp3)
    b2 = A * ((A + 1) - temp2 - temp1)
    a0 = (A + 1) + temp2 + temp1
    a1 = -2 * ((A - 1) + temp3)
    a2 = (A + 1) + temp2 - temp1
    return [b0, b1, b2, a0, a1, a2]
    #return [b0/a0, b1/a0, b2/a0, 1, a1/a0, a2/a0]

In [28]:
central_freq = 1000
#q = 0.707
gain = 40

'''
width determines how steep is the filter’s shelf transition. 
In addition to the common width specification methods described above, 
‘slope’ (the default, or if appended with ‘s’) may be used. 
The useful range of ‘slope’ is about 0.3, for a gentle slope, to 1 (the maximum), 
for a steep slope; the default value is 0.5. 
'''
s = 1 

A = math.exp(gain / 40 * math.log(10))
q = 1/math.sqrt(2 + (A + 1/A) * (1/s - 1))
print(q)
filename = 'whitenoise.wav'
noise_filepath = common_utils.get_asset_path(filename)

# input q in sox
E = torchaudio.sox_effects.SoxEffectsChain()
E.set_input_file(noise_filepath)
E.append_effect_to_chain("bass", [gain, central_freq, str(q) + 'q'])
sox_output_waveform_q, sr = E.sox_build_flow_effects()

# input s in sox
E = torchaudio.sox_effects.SoxEffectsChain()
E.set_input_file(noise_filepath)
E.append_effect_to_chain("bass", [gain, central_freq, str(s) + 's'])
sox_output_waveform_s, sr = E.sox_build_flow_effects()

# input biquad coefficients from our code in sox
coeff = bass_biquad_coeff(waveform, sample_rate, gain, central_freq, q)

E = torchaudio.sox_effects.SoxEffectsChain()
E.set_input_file(noise_filepath)
E.append_effect_to_chain("biquad", coeff)
sox_output_waveform_biquad, sr = E.sox_build_flow_effects()

# Above three output waveforms are strictly same !

waveform, sample_rate = torchaudio.load(noise_filepath, normalization=True)
output_waveform = bass_biquad(waveform, sample_rate, gain, central_freq, q)

# Normalized and Unnormalized coefficients
[b0, b1, b2, a0, a1, a2] = bass_biquad_coeff(waveform, sample_rate, gain, central_freq, q)
print(b0/a0, b1/a0, b2/a0, 1, a1/a0, a2/a0)
output_waveform1 = F.biquad(waveform, b0, b1, b2, a0, a1, a2)
output_waveform2 = F.biquad(waveform, b0/a0, b1/a0, b2/a0, 1, a1/a0, a2/a0)

# these two outputs are different! Compared to SoX result, the second result is better (max error = 1.4e-4).


0.7071067811865475
1.3270219250069244 -1.8385497216920241 0.7088138842796521 1 -1.9362063350513272 0.9381791959272727


In [29]:
atol = 1e-10
diff = abs(sox_output_waveform_biquad - sox_output_waveform_s)
#diff = abs(sox_output_waveform_biquad - sox_output_waveform_q)

print((diff > atol).sum())

tensor(0)


In [30]:
atol = 1e-4
diff = abs(output_waveform1 - output_waveform2)
print((diff > atol).sum())

tensor(12603)


In [31]:
atol = 1.5e-4
diff = abs(sox_output_waveform_biquad - output_waveform2)
print((diff > atol).sum())

tensor(0)


This test shows that the coeffients input to SoX and bass_biquad function are same. Nomalized coefficients introduce smaller error for bass filter. To double check this, biquad is calculated by lfilter function. I test the result of lfilter function again as below.

### 4. Compare result with and without coefficients normalization.

In [32]:
[b0, b1, b2, a0, a1, a2] = bass_biquad_coeff(waveform, sample_rate, gain, central_freq, q)
device = waveform.device
dtype = waveform.dtype

output_waveform1 = F.lfilter(
        waveform,
        torch.tensor([a0, a1, a2], dtype=dtype, device=device),
        torch.tensor([b0, b1, b2], dtype=dtype, device=device),
        clamp = True
    )

output_waveform2 = F.lfilter(
        waveform,
        torch.tensor([a0/a0, a1/a0, a2/a0], dtype=dtype, device=device),
        torch.tensor([b0/a0, b1/a0, b2/a0], dtype=dtype, device=device),
        clamp = True

    )


atol = 1e-4
diff = abs(output_waveform2 - output_waveform1)
print((diff > atol).sum())

tensor(12603)


In [33]:
atol = 1.5e-4
diff = abs(output_waveform2 - sox_output_waveform_s)
print((diff > atol).sum())

tensor(0)


### 5. Compare output waveforms between SoX, code and Matlab toolbox

In [34]:

'''
Parameters for SoX, code and matlab toolbox:
central_freq = 1000
gain = 40
s = 1 
A = math.exp(gain / 40 * math.log(10))
q = 1/math.sqrt(2 + (A + 1/A) * (1/s - 1)) 
# q = 0.707
'''
# Find first 8 elements with largest errors
value, index = torch.topk(abs(sox_output_waveform_biquad-output_waveform2), 8)
print(sox_output_waveform_biquad[0, index])
print(output_waveform2[0, index])
print(sox_output_waveform_biquad[0, index] - output_waveform2[0, index])

tensor([[-0.9010, -0.9252, -0.8271, -0.5863, -0.9767,  0.2632, -0.9477, -0.9332]])
tensor([[-0.9008, -0.9253, -0.8273, -0.5864, -0.9768,  0.2633, -0.9478, -0.9333]])
tensor([[-0.0001,  0.0001,  0.0001,  0.0001,  0.0001, -0.0001,  0.0001,  0.0001]])


In [35]:
# Compare three output waveforms for gain = 10 db
# Firstly change gain to an example of 10 db because the max gain is 12 db in matlab toobox.

'''
gain = 10
'''

value, index = torch.topk(abs(sox_output_waveform_biquad-output_waveform2), 8)
print(sox_output_waveform_biquad[0, index])
print(output_waveform2[0, index])
print(sox_output_waveform_biquad[0, index] - output_waveform2[0, index])

print(output_waveform2[0, 0:8], sox_output_waveform_biquad[0, 0:8])

matlab_res = [0.1482055675, 0.11175017674, -0.48174034477, -0.75495056776, 0.414173566182,
             0.56533516912, 0.60206182193, 0.767192813403, 0.377428142554, 0.328867691928]

# check if result of matlab toolbox close to result of sox or my code
index_check = 1
print(abs(output_waveform2[0, index_check] - matlab_res[index_check]) > abs(sox_output_waveform_biquad[0, index_check] - matlab_res[index_check]))

tensor([[-0.9010, -0.9252, -0.8271, -0.5863, -0.9767,  0.2632, -0.9477, -0.9332]])
tensor([[-0.9008, -0.9253, -0.8273, -0.5864, -0.9768,  0.2633, -0.9478, -0.9333]])
tensor([[-0.0001,  0.0001,  0.0001,  0.0001,  0.0001, -0.0001,  0.0001,  0.0001]])
tensor([-0.1855,  0.0592, -0.6334, -1.0000, -0.2192,  0.1155,  0.4404,  0.9970]) tensor([-0.1855,  0.0592, -0.6334, -1.0000, -0.2192,  0.1155,  0.4404,  0.9970])
tensor(True)


The code result matches with SoX and matlab code. The error increases when gain increases ( 1.4e-4 at edge case: gain = 40, slope = 1, Q = 0.707, fc = 1000).

### 6. Test other filters with normalized coefficients

In [36]:
def treble_biquad(
        waveform: Tensor,
        sample_rate: int,
        gain: float,
        central_freq: float = 3000,
        Q: float = 0.707
) -> Tensor:
    r"""Design a treble tone-control effect.  Similar to SoX implementation.
    Args:
        waveform (Tensor): audio waveform of dimension of `(..., time)`
        sample_rate (int): sampling rate of the waveform, e.g. 44100 (Hz)
        gain (float): desired gain at the boost (or attenuation) in dB.
        central_freq (float, optional): central frequency (in Hz). (Default: ``3000``)
        Q (float, optional): https://en.wikipedia.org/wiki/Q_factor (Default: ``0.707``).
    Returns:
        Tensor: Waveform of dimension of `(..., time)`
    References:
        http://sox.sourceforge.net/sox.html
        https://www.w3.org/2011/audio/audio-eq-cookbook.html#APF
    """
    w0 = 2 * math.pi * central_freq / sample_rate
    alpha = math.sin(w0) / 2 / Q
    A = math.exp(gain / 40 * math.log(10))

    temp1 = 2 * math.sqrt(A) * alpha
    temp2 = (A - 1) * math.cos(w0)
    temp3 = (A + 1) * math.cos(w0)

    b0 = A * ((A + 1) + temp2 + temp1)
    b1 = -2 * A * ((A - 1) + temp3)
    b2 = A * ((A + 1) + temp2 - temp1)
    a0 = (A + 1) - temp2 + temp1
    a1 = 2 * ((A - 1) - temp3)
    a2 = (A + 1) - temp2 - temp1
    return F.biquad(waveform, b0, b1, b2, a0, a1, a2)

    #return F.biquad(waveform, b0/a0, b1/a0, b2/a0, a0/a0, a1/a0, a2/a0)

In [37]:
central_freq = 1000
q = 0.707
gain = 40
#s = 0.5
#filename = 'steam-train-whistle-daniel_simon.mp3'
#filename = 'sinewave.wav'
filename = 'whitenoise.wav'
noise_filepath = common_utils.get_asset_path(filename)
E = torchaudio.sox_effects.SoxEffectsChain()
E.set_input_file(noise_filepath)
E.append_effect_to_chain("treble", [gain, central_freq, str(q) + 'q'])
sox_output_waveform, sr = E.sox_build_flow_effects()

waveform, sample_rate = torchaudio.load(noise_filepath, normalization=True)
output_waveform = treble_biquad(waveform, sample_rate, gain, central_freq, q)
#self.assertEqual(output_waveform, sox_output_waveform, atol=1e-4, rtol=1e-5)

In [38]:
atol = 5e-5
diff = abs(output_waveform - sox_output_waveform)
print((diff > atol).sum())

tensor(2)


Results are close with and without coefficients nomalization for treble filter